# 기본환경 설정

In [ ]:
# !pip install faiss-cpu

In [ ]:
from google.colab import userdata
HF_KEY = userdata.get("HF_KEY")

In [ ]:
import huggingface_hub
huggingface_hub.login(HF_KEY)

# 모델 로딩

In [ ]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo langchain-community pypdf langchain_huggingface faiss-cpu
!pip install --no-deps unsloth

In [ ]:
from unsloth import FastModel
from langchain.embeddings import HuggingFaceEmbeddings
import torch

In [ ]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [ ]:
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 1024*5, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    # device_map = {"": device}
)

In [ ]:
model = FastModel.for_inference(model)

# Custom ChatModel 함수

In [ ]:
from typing import Any, Dict, Optional, List, Union, Type, Literal
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, BaseMessage
from langchain_core.outputs import ChatGeneration, ChatResult

In [ ]:
from langchain_core.runnables import RunnableLambda
from typing import Any, Dict, Optional, List, Union, Type, Literal
import json, warnings
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage

In [ ]:
from pydantic import BaseModel as PydanticBaseModel

In [ ]:
class GemmaChatModel(BaseChatModel):
    def __init__(self, model, tokenizer, max_tokens: int = 512, do_sample: bool = True, temperature: float = 0.7, top_p: float = 0.9):
        super().__init__()
        object.__setattr__(self, "model", model)
        object.__setattr__(self, "tokenizer", tokenizer)
        object.__setattr__(self, "max_tokens", max_tokens)
        object.__setattr__(self, "do_sample", do_sample)
        object.__setattr__(self, "temperature", temperature)
        object.__setattr__(self, "top_p", top_p)

    @property
    def _llm_type(self) -> str:
        return "gemma-chat"

    def _format_messages(self, messages: List[BaseMessage]) -> str:
        prompt = ""
        for m in messages:
            if isinstance(m, SystemMessage):
                prompt += f"<|system|>\n{m.content}</s>\n"
            elif isinstance(m, HumanMessage):
                prompt += f"<|user|>\n{m.content}</s>\n"
            elif isinstance(m, AIMessage):
                prompt += f"<|assistant|>\n{m.content}</s>\n"
        prompt += "<|assistant|>\n"
        return prompt

    def _apply_stop(self, text: str, stop: Optional[List[str]]) -> str:
        if not stop:
            return text
        cut = len(text)
        for s in stop:
            idx = text.find(s)
            if idx != -1:
                cut = min(cut, idx)
        return text[:cut]

    def _generate(self, messages: List[BaseMessage], stop: Optional[List[str]] = None, run_manager: Optional[Any] = None, **kwargs: Any) -> ChatResult:
        prompt = self._format_messages(messages)
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

        gen_kwargs = {
            "max_new_tokens": kwargs.get("max_tokens", self.max_tokens),
            "do_sample": kwargs.get("do_sample", self.do_sample),
            "temperature": kwargs.get("temperature", self.temperature),
            "top_p": kwargs.get("top_p", self.top_p),
            "eos_token_id": self.tokenizer.eos_token_id,
            "pad_token_id": self.tokenizer.pad_token_id,
        }

        with torch.no_grad():
            outputs = self.model.generate(**inputs, **gen_kwargs)

        decoded = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        # 마지막 assistant 턴 이후만 추출
        if "<|assistant|>\n" in decoded:
            response = decoded.split("<|assistant|>\n")[-1]
        else:
            response = decoded
        response = response.strip()
        response = self._apply_stop(response, stop)

        return ChatResult(generations=[ChatGeneration(message=AIMessage(content=response))])

    ### Structured output support #######################################################################################
    def _build_json_system_prompt(self, schema_text: str) -> str:
        # 모델이 JSON만 내도록 강하게 지시 (hallucination 방지용 규칙 포함)
        return (
            "You are a strict JSON generator.\n"
            "Return ONLY a single JSON object, no prose, no backticks, no explanations.\n"
            "Do not include trailing commas. Do not include comments.\n"
            "Conform exactly to the following JSON schema (fields, types, required):\n"
            f"{schema_text}\n"
        )

    def _ensure_pydantic(self):
        if PydanticBaseModel is None:
            raise RuntimeError(
                "Pydantic is not available. Install pydantic or pass a dict schema instead of a BaseModel."
            )

    def _schema_to_text(self, schema: Union[Type["PydanticBaseModel"], Dict[str, Any]]) -> str:
        if PydanticBaseModel is not None and isinstance(schema, type) and issubclass(schema, PydanticBaseModel):
            # pydantic 스키마를 JSON 스키마로 직렬화
            try:
                # v1/v2 호환 직렬화
                json_schema = schema.model_json_schema()  # pydantic v2
            except Exception:
                json_schema = schema.schema()  # pydantic v1
            return json.dumps(json_schema, ensure_ascii=False, indent=2)
        elif isinstance(schema, dict):
            return json.dumps(schema, ensure_ascii=False, indent=2)
        else:
            raise TypeError("schema must be a Pydantic BaseModel subclass or a dict JSON schema.")

    def _parse_structured(self, text: str, schema: Union[Type["PydanticBaseModel"], Dict[str, Any]], include_raw: bool):
        # 코드블록 등 제거 시도(혹시 들어올 경우)
        t = text.strip()
        if t.startswith("```"):
            # ```json ... ``` 또는 ``` ... ```
            t = t.strip("`")
            # 첫 줄에 json 명시가 들어있을 수 있음
            t = "\n".join(line for line in t.splitlines() if not line.lower().startswith("json"))
        # JSON 파싱
        obj = json.loads(t)

        # Pydantic 검증
        if PydanticBaseModel is not None and isinstance(schema, type) and issubclass(schema, PydanticBaseModel):
            validated = schema.model_validate(obj) if hasattr(schema, "model_validate") else schema.parse_obj(obj)
            return {"parsed": validated, "raw": text} if include_raw else validated
        else:
            # dict 스키마는 별도 검증 없이 반환 (원하면 jsonschema로 검증 가능)
            return {"parsed": obj, "raw": text} if include_raw else obj

    def with_structured_output(
        self,
        schema: Union[Type["PydanticBaseModel"], Dict[str, Any]],
        *,
        method: Literal["json_mode"] = "json_mode",
        include_raw: bool = False,
        system_prefix: Optional[str] = None,
        deterministic: bool = True,
    ):
        schema_text = self._schema_to_text(schema)
        sys_prompt = self._build_json_system_prompt(schema_text)
        if system_prefix:
            sys_prompt = system_prefix.rstrip() + "\n\n" + sys_prompt

        def _invoke(messages_or_any):
            # 입력을 메시지 리스트로 정규화
            if isinstance(messages_or_any, list) and all(isinstance(m, BaseMessage) for m in messages_or_any):
                msgs = [SystemMessage(content=sys_prompt)] + messages_or_any
            else:
                # 문자열/딕셔너리 등도 처리
                msgs = [SystemMessage(content=sys_prompt), HumanMessage(content=str(messages_or_any))]

            # 결정론 옵션
            kw = {}
            if deterministic:
                kw = {"do_sample": False, "temperature": 0.0, "top_p": 1.0}

            # 1차 시도
            result = self._generate(msgs, **kw)
            text = result.generations[0].message.content
            try:
                return self._parse_structured(text, schema, include_raw)
            except Exception:
                # 재시도: 더 강한 지시
                retry_msgs = [SystemMessage(content=sys_prompt + "\nOutput must be valid JSON. Try again.")] + msgs[1:]
                result2 = self._generate(retry_msgs, **kw)
                text2 = result2.generations[0].message.content
                return self._parse_structured(text2, schema, include_raw)

        # Runnable 로 래핑해서 반환 (체인 파이프에 바로 사용 가능)
        return RunnableLambda(_invoke)

In [ ]:
chat_model = GemmaChatModel(model=model, tokenizer=tokenizer, max_tokens=1024*5)

# Document Load

In [ ]:
from langchain_community.document_loaders import GitLoader

In [ ]:
def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")

In [ ]:
loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

In [ ]:
raw_docs = loader.load()
print(len(raw_docs))

# Document Transformer

## CharacterTextSplitter
- 사용자가 separator 인자로 지정한 단일 문자(예: "\n")을 기준으로 분할합니다.
- 사용자가 분할 기준으로 직접 정할 수도 있습니다.

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

In [ ]:
text_splitter = CharacterTextSplitter(
    chunk_size=1000, # 목표치
    chunk_overlap=0,
    separator="\n\n", #사용자가 직접 지정 가능
)

In [ ]:
chunk_docs = text_splitter.split_documents(raw_docs)
print(len(chunk_docs))

In [ ]:
print(f"청크 개수: {len(chunk_docs)}")

for i, doc in enumerate(chunk_docs[:2]):
    print(f"\n--- 조각 {i+1} ({len(doc.page_content)}) ---\n{doc.page_content}")

## RecursiveCharacterTextSplitter
- ["\n\n", "\n", " ", ""] 순서로, 큰 단위(문단)부터 작은 단위(단어)로 재귀적으로 분할합니다.
- "" : 문자(character)단위로 분할

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", " "],  # 여러 구분자를 순차적으로 적용
    chunk_size=1000,
    chunk_overlap=10,
)

In [ ]:
chunk_docs = text_splitter.split_documents(raw_docs)

In [ ]:
print(f"청크 개수: {len(chunk_docs)}")

for i, doc in enumerate(chunk_docs[:2]):
    print(f"\n--- 조각 {i+1} ({len(doc.page_content)}) ---\n{doc.page_content}")

## TokenTextSplitter
- 언어 모델의 토큰 단위로 chunk_size 길이에 맞추어 분할합니다.

In [ ]:
from langchain.text_splitter import TokenTextSplitter

In [ ]:
# openai 토그나이저로 자르기
text_splitter = TokenTextSplitter(
    model_name="gpt-3.5-turbo",
    chunk_size=1000,
    chunk_overlap=0
)

In [ ]:
chunk_docs = text_splitter.split_documents(raw_docs)

In [ ]:
print(f"청크 개수: {len(chunk_docs)}")

for i, doc in enumerate(chunk_docs[:2]):
    print(f"\n--- 조각 {i+1} ({len(doc.page_content)}) ---\n{doc.page_content}")

In [ ]:
# Local Tokenizer로 자르기
text_splitter = TokenTextSplitter.from_huggingface_tokenizer(
    tokenizer.tokenizer,  # Gemma3Processor 객체 안에서 토크나이저를 찾아 넣어야 함.
    chunk_size=1000,
    chunk_overlap=0
) 

In [ ]:
# Local Tokenizer로 자르기 : 문맥을 좀더 효율적으로 찾아서 분리.
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer.tokenizer,
    chunk_size=1000,
    chunk_overlap=0,             # 필요시 100~200 정도로 조정
    add_start_index=False        # 청크 시작 인덱스가 필요하면 True
)

## NLTKTextSplitter

내부적으로 nltk.sent_tokenize() → Punkt 문장 분리기 사용  
즉, 문장 경계 후보(., ?, !, …) + Punkt 규칙(약어/숫자/대문자 등)을 기반으로 문장을 나눔  

* 기호 후보: ., ?, !, …는 잠정적 문장 끝 후보
* 대문자 시작: 구분 기호 뒤 단어가 대문자로 시작 → 문장 끝일 가능성 ↑
* 약어 인식: Dr., Mr., e.g. 등은 문장 끝이 아님
* 이니셜/다중 점: U.S.A., J. R. R. 같은 패턴은 내부로 처리
* 숫자/소수점: 3.14, No. 5 같은 경우는 문장 끝 아님
* 엘립시스(...): 상황에 따라 문장 끝일 수도 있고 아닐 수도 있음
* 따옴표/괄호: .", !) 같은 닫는 부호는 함께 문장 끝으로 취급
* 특수 패턴: URL, 이메일, 파일명은 경계로 오인하지 않음  

In [ ]:
from langchain_text_splitters import NLTKTextSplitter

In [ ]:
import nltk, os

download_dir = os.path.join(os.getcwd(), 'nltk_data')
nltk.download('punkt', download_dir=download_dir)
nltk.download('punkt_tab', download_dir=download_dir)
nltk.data.path.append(download_dir)

In [ ]:
text_splitter = NLTKTextSplitter(
    chunk_size=1000,
    chunk_overlap=0
)

In [ ]:
chunk_docs = text_splitter.split_documents(raw_docs)

In [ ]:
print(f"청크 개수: {len(chunk_docs)}")

for i, doc in enumerate(chunk_docs[:2]):
    print(f"\n--- 조각 {i+1} ({len(doc.page_content)}) ---\n{doc.page_content}")

## MarkdownTextSplitter
- 마크다운 텍스트를 구조적 요소(헤더(#), 리스트(-, *), 코드 블록(```))로, chunk_size 길이에 맞추어 분할합니다.

In [ ]:
from langchain.text_splitter import MarkdownTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter

### Markdown-Based Splitting
크기를 우선으로 텍스트를 분할 -> 마크다운 문법을 존중하며 지정된 chunk_size를 넘지 않도록 분할

In [ ]:
text_splitter = MarkdownTextSplitter(
    chunk_size=1000,
    chunk_overlap=0
)

In [ ]:
chunk_docs = text_splitter.split_documents(raw_docs)

In [ ]:
print(f"청크 개수: {len(chunk_docs)}")

for i, doc in enumerate(chunk_docs[:2]):
    print(f"\n--- 조각 {i+1} ({len(doc.page_content)}) ---\n{doc.page_content}")

### Markdown Header-Based Splitting
헤더( #, ##)에 따른 구조적 조각 생성 -> headers_to_split_on에 지정된 헤더 태그가 나타날 때마다 분할

In [ ]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

In [ ]:
text_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)

In [ ]:
chunk_docs = text_splitter.split_text(raw_docs[0].page_content)

In [ ]:
print(f"청크 개수: {len(chunk_docs)}")

for i, doc in enumerate(chunk_docs[:2]):
    print(f"\n--- 조각 {i+1} ({len(doc.page_content)}) --- {doc.metadata}\n\n{doc.page_content}")

## HTMLHeaderTextSplitter
- HTML 문서를 \<h1>, \<h2> 등 Header 태그 기준으로 분할합니다. chunk_size, chunk_overlap 파라미터를 받지 않습니다.

In [ ]:
from langchain.text_splitter import HTMLHeaderTextSplitter

In [ ]:
with open('./res/ICSA.html', 'r', encoding='utf-8') as f:
    html_text = f.read()

In [ ]:
headers_to_split_on = [
    ("h1", "Title"),
    ("h2", "Section"),
    ("h3", "Subsection"),
    ("h4", "Details")
]

In [ ]:
text_splitter = HTMLHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)

In [ ]:
chunk_docs = text_splitter.split_text(html_text)

In [ ]:
print(f"청크 개수: {len(chunk_docs)}")

for i, doc in enumerate(chunk_docs[:2]):
    print(f"\n--- 조각 {i+1} ({len(doc.page_content)}) ---\n{doc.page_content}")

## LanguageSpecificTextSplitter
- class, def (Python의 경우) 등 선택한 프로그래밍 언어의 주요 구문(클래스, 함수 정의 등)을 기준으로 분할합니다.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

In [ ]:
with open('res/sample_python.py', 'r', encoding='utf-8') as f:
    python_code = f.read()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, 
    chunk_size=400, 
    chunk_overlap=0
)

In [ ]:
chunk_docs = text_splitter.split_text(python_code)

In [ ]:
print(f"청크 개수: {len(chunk_docs)}")

for i, doc in enumerate(chunk_docs[:10]):
    print(f"\n--- 조각 {i+1} ({len(doc)}) ---\n{doc}")

# Test Splitter 적용

In [ ]:
text_splitter = MarkdownTextSplitter(
    chunk_size=1000,
    chunk_overlap=0
)

In [ ]:
docs = text_splitter.split_documents(raw_docs)

# Embedding

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
MODEL_EMBED = "intfloat/multilingual-e5-base" # sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
embedding = HuggingFaceEmbeddings(model_name=MODEL_EMBED)

In [ ]:
query = "AWS의 S3에서 데이터를 읽어 들이기 위한 Document loader가 있나요?"

vector = embedding.embed_query(query)
print(len(vector))
print(vector)

# Vector store

In [ ]:
from langchain_chroma import Chroma

In [ ]:
db = Chroma.from_documents(docs, embedding)

In [ ]:
retriever = db.as_retriever()

In [ ]:
query = "AWS의 S3에서 데이터를 읽어 들이기 위한 Document loader가 있나요?"

context_docs = retriever.invoke(query)
print(f"len = {len(context_docs)}")

In [ ]:
first_doc = context_docs[0]
print(f"metadata = {first_doc.metadata}")
print(first_doc.page_content)

# LCEL을 사용한 RAG 구현

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_template('''
다음 문맥만을 바탕으로 질문에 답변해 주세요.

문맥: """
{context}
"""

질문: {question}
''')

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | chat_model
    | StrOutputParser()
)

In [ ]:
output = chain.invoke(query)
print(output)